In [1]:
import pandas as pd
import numpy as np
import torch
import torch.utils.data
import torch.nn as nn
from torch.autograd import Variable
from sklearn.model_selection import train_test_split
import joblib
from tqdm import tqdm
import matplotlib.pyplot as plt
from copy import deepcopy

In [2]:
from isaac.utils import get_cuda_device_if_available, create_directory
device = get_cuda_device_if_available()
print(device)

data_directory = "multibranch_plots/"
create_directory(data_directory)
create_directory("models")
create_directory("scalers")

cuda:0


In [3]:
from isaac.dataset import read_dataset, prepare_dataset
from isaac.utils import plot_confusion_matrix
from isaac.constants import BASIC_TRAINING_COLS, FORCE_CLASS_COLS, MASS_CLASS_COLS
from isaac.training import training_loop
from isaac.models import MultiBranchModel, initialise_model
from isaac.evaluation import get_best_model_and_its_accuracy

In [4]:
BATCH_SIZE = 128
EPOCHS = 2
NORMALISE_DATA = True
STEP_SIZE = 3
SEQ_END = 1800

INPUT_DIM = len(BASIC_TRAINING_COLS)    # input dimension
HIDDEN_DIM = 25  # hidden layer dimension
N_LAYERS = 4     # number of hidden layers
OUTPUT_DIM = 3   # output dimension
DROPOUT = 0.5

network_params = (INPUT_DIM, HIDDEN_DIM, OUTPUT_DIM, DROPOUT)

## Read dataset and preprocess it

In [5]:
train_trials = read_dataset("data/train_passive_trials.h5", n_trials=10)
val_trials = read_dataset("data/val_passive_trials.h5", n_trials=10)

100%|██████████| 10/10 [00:00<00:00, 115.99it/s]


## Define model, loss and optimizer

## Train model and plot loss and accuracy

In [6]:
stats_dfs = []
loaders, scaler = prepare_dataset([train_trials, val_trials], 
                                  class_columns=[list(MASS_CLASS_COLS), list(FORCE_CLASS_COLS)], 
                                  multiclass=True,
                                  batch_size=BATCH_SIZE, normalise_data=NORMALISE_DATA,
                                  device=device)

best_mass_model_overall = None
best_mass_accuracy = 0.
best_force_model_overall = None
best_force_accuracy = 0.

for seed in [0, 42, 72]:
    df = pd.DataFrame()


    model, error, optimizer = initialise_model(network_params, lr=0.01, seed=seed, device=device, arch=MultiBranchModel)
    epoch_losses, epoch_accuracies, [best_mass_model, best_force_model] = training_loop(model, optimizer, 
                                                                                        error,
                                                                                        loaders[0], loaders[1], 
                                                                                        EPOCHS, seq_end=SEQ_END,
                                                                                        step_size=STEP_SIZE,
                                                                                        multibranch=True)

    train_accuracies = np.array(epoch_accuracies[0])
    val_accuracies = np.array(epoch_accuracies[1])
    
    best_mass_model_overall, best_mass_accuracy = get_best_model_and_its_accuracy(best_mass_model,
                                                                                  max(val_accuracies[:, 0]), 
                                                                                  best_mass_model_overall,
                                                                                  best_mass_accuracy)

    best_force_model_overall, best_force_accuracy = get_best_model_and_its_accuracy(best_force_model,
                                                                                    max(val_accuracies[:, 1]), 
                                                                                    best_force_model_overall,
                                                                                    best_force_accuracy)
    
    df["Epoch"] = np.arange(EPOCHS)
    df["Mass Loss"] = epoch_losses[:, 0]
    df["Force Loss"] = epoch_losses[:, 1]
    df["Mass Train Accuracy"] = train_accuracies[:, 0]
    df["Mass Val Accuracy"] = val_accuracies[:, 0]
    df["Force Train Accuracy"] = train_accuracies[:, 1]
    df["Force Val Accuracy"] = val_accuracies[:,1]
    df["seed"] = str(seed)
    stats_dfs.append(df)
        
stats = pd.concat(stats_dfs)
stats.to_hdf(data_directory+"stats.h5", key="stats")

100%|██████████| 10/10 [00:00<00:00, 598.27it/s]
Train_loss: ([1.07427418 1.06942928]) Train_acc: ([80. 60.]) Val_acc: ([20. 40.]): 100%|██████████| 2/2 [00:00<00:00,  3.91it/s]
Train_loss: ([1.06175053 1.02226663]) Train_acc: ([40. 70.]) Val_acc: ([40. 20.]): 100%|██████████| 2/2 [00:00<00:00,  5.18it/s]
Train_loss: ([1.05432892 1.04022336]) Train_acc: ([50. 40.]) Val_acc: ([30. 30.]): 100%|██████████| 2/2 [00:00<00:00,  5.12it/s]


## Save model and scaler

In [7]:
best_mass_accuracy, best_force_accuracy

(40.0, 40.0)

In [8]:
torch.save(best_mass_model_overall.state_dict(), "models/passive_mass_dual_model.pt")
torch.save(best_force_model_overall.state_dict(), "models/passive_force_dual_model.pt")

In [9]:
joblib.dump(scaler, "scalers/passive_dual_scaler.sk")

['scalers/passive_dual_scaler.sk']